Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [39]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [40]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [41]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [42]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
# fundsSI

In [43]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,EUCA4,14.61,50.730169,137.0,2.472291,4.30,26.60,3.39
2,BRSR6,11.25,36.772279,136.0,2.268647,2.46,24.43,4.58
3,ALLD3,8.00,25.177177,135.0,2.147147,1.67,16.87,4.80
4,MTRE3,3.83,11.970620,134.0,2.125488,0.69,9.23,5.58
5,SOMA3,5.95,18.293851,133.0,2.074597,2.01,7.40,2.96
6,BMGB4,3.43,9.531317,132.0,1.778810,0.56,7.21,6.13
7,JHSF3,4.03,11.172735,131.0,1.772391,0.76,7.30,5.29
8,GOAU4,9.99,26.214576,130.0,1.624082,1.68,18.18,5.94
9,GOAU3,10.21,26.214576,129.0,1.567539,1.68,18.18,6.07
10,NEOE3,18.08,46.370821,128.0,1.564758,3.67,26.04,4.92


In [44]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,14.61,50.730169,137.0,2.472291,4.30,26.60
2,BRSR6,11.25,36.772279,136.0,2.268647,2.46,24.43
3,ALLD3,8.00,25.177177,135.0,2.147147,1.67,16.87
4,MTRE3,3.83,11.970620,134.0,2.125488,0.69,9.23
5,SOMA3,5.95,18.293851,133.0,2.074597,2.01,7.40
6,BMGB4,3.43,9.531317,132.0,1.778810,0.56,7.21
7,JHSF3,4.03,11.172735,131.0,1.772391,0.76,7.30
8,GOAU4,9.99,26.214576,130.0,1.624082,1.68,18.18
10,NEOE3,18.08,46.370821,128.0,1.564758,3.67,26.04
11,SCAR3,22.50,56.544708,127.0,1.513098,4.84,29.36


outros criterios adicionais

In [45]:
df_limpo =  df_limpo[(fundsSI['P/L'] > 0)]
top20 = df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,14.61,50.730169,137.0,2.472291,4.30,26.60
2,BRSR6,11.25,36.772279,136.0,2.268647,2.46,24.43
3,ALLD3,8.00,25.177177,135.0,2.147147,1.67,16.87
4,MTRE3,3.83,11.970620,134.0,2.125488,0.69,9.23
5,SOMA3,5.95,18.293851,133.0,2.074597,2.01,7.40
6,BMGB4,3.43,9.531317,132.0,1.778810,0.56,7.21
7,JHSF3,4.03,11.172735,131.0,1.772391,0.76,7.30
8,GOAU4,9.99,26.214576,130.0,1.624082,1.68,18.18
10,NEOE3,18.08,46.370821,128.0,1.564758,3.67,26.04
11,SCAR3,22.50,56.544708,127.0,1.513098,4.84,29.36


In [46]:
df_limpo =  df_limpo[(fundsSI['DIVIDA LIQUIDA / EBIT'] < 3.1) ]
top20 = df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,EUCA4,14.61,50.730169,137.0,2.472291,4.30,26.60
3,ALLD3,8.00,25.177177,135.0,2.147147,1.67,16.87
5,SOMA3,5.95,18.293851,133.0,2.074597,2.01,7.40
7,JHSF3,4.03,11.172735,131.0,1.772391,0.76,7.30
8,GOAU4,9.99,26.214576,130.0,1.624082,1.68,18.18
11,SCAR3,22.50,56.544708,127.0,1.513098,4.84,29.36
12,MELK3,3.48,8.616844,126.0,1.476105,0.55,6.00
13,BRAP3,17.16,41.937275,125.0,1.443897,3.74,20.90
15,SAPR3,5.33,12.460839,123.0,1.337868,1.03,6.70
19,VLID3,16.98,39.507847,119.0,1.326728,3.69,18.80


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.